# Retrieving Slack Conversations with Slack API 

In [1]:
!pip install slack_sdk

Create a little helper class to simplify the usage of Slack SDK for our purposes.

In [13]:
import datetime
import time
import os
from slack_sdk import WebClient

class SlackHelper:

    def __init__(self, token: str):
        self._username_cache = dict()
        self._slack_client = WebClient(token)
        self._slack_client.api_test()

    def print_messages_from_channel(self, channel):
        messages = self._slack_client.conversations_history(channel=channel)["messages"]
        for message in reversed(messages):
            message_ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(float(message["ts"])))
            print("(" + message_ts + ", user=" + self.get_user_name(message["user"]) + "): " + message["text"])
            if "thread_ts" in message:
                replies = self._slack_client.conversations_replies(channel=channel, ts=message["thread_ts"])["messages"]
                for reply in replies[1:]:
                    reply_ts = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(float(message["ts"])))
                    print("      (" + reply_ts + ", user=" + self.get_user_name(reply["user"]) + "): " + reply["text"])

    def get_user_name(self, user):
        if user not in self._username_cache:
            self._username_cache[user] = self._slack_client.users_info(user=user)["user"]["real_name"]
        return self._username_cache[user]

    def get_conversations_info(self, channel):
        response = self._slack_client.conversations_info(channel=channel, include_num_members=1)
        return response["channel"]

    def list_channels(self):
        response = self._slack_client.conversations_list(types="public_channel, private_channel, mpim, im")
        return response["channels"]

    def get_channel_by_name(self, channel_name):
        for channel in self.list_channels():
            if channel['name'] == channel_name:
                return channel['id']
        return None

We need a Slack API Token for a custom app. You can create one [https://api.slack.com/apps/](here). Select `Create New App`->`
From scratch` and provide an app name and a workspace where you develop the app. This needs to be a personal Slack workspace as you don't have permissions to create apps in IBM Slack workspaces.

Now set up a bot token for your app in your workspace. In the navigation pane to the left select `Features`->`OAuth & Permissions`. Scroll down to section `Scopes` and add scopes for `app_mentions:read`, `channels:history`, `channels:read`, `groups:history`, `groups:read`,  `im:read`, `mpim:read` and `users:read`.
Now scroll up to section `OAuth Tokens for Your Workspace` click `Install to Workspace`. Now select your workspace where the app should be deployed (again note: You can't deploy to IBM Slack workspaces yourselfes. This requires an IBM Slack admin). Slack generates a new `Bot User OAuth Token` as part of this workspace deployment. Copy this token. You need it to run the below logic.

You now need to add the new app to the channel(s) for which you want to retrieve the conversations. To do so, go to your Slack client type `@<your app name>` and press enter. Slack opens a dialog asking you whether you want to `Add to channel`. Click this option to add the app bot to your channel. 

You are now set up to proceed with the notebook.

Set the Slack API Token.

In [14]:
try:
    slack_api_token = os.environ["SLACK_API_TOKEN"]
except KeyError:
    slack_api_token = getpass.getpass("Please enter your Slack API Token (hit enter): ")

Initialize our Slack helper class.

In [15]:
slack = SlackHelper(slack_api_token)

Set the slack channel.

In [17]:
try:
    slack_channel_name = os.environ["SLACK_CHANNEL"]
except KeyError:
    slack_channel_name = input("Please enter the name of youz Slack channel to work with (hit enter): ")

slack_channel = slack.get_channel_by_name("foo")
if not slack_channel:
    raise BaseException("Channel " + slack_channel_name + " does not exist")
print("Your slack channel is #" + slack_channel_name + " with channel ID " + slack_channel + ".")

Your slack channel is #foo with channel ID C05RK7JB336.


Retrieve the full conversation history of the selected channel.

In [5]:
slack.print_messages_from_channel(slack_channel)

(2023-09-08 06:59:17, user=torsten): <@U05S5PHRU7J> has joined the channel
(2023-09-08 07:22:06, user=torsten): Moin Moin!!
(2023-09-08 07:30:01, user=Torsten): <@U05S5QZF6TA> has joined the channel
(2023-09-08 07:36:07, user=torsten): Funzt!
      (2023-09-08 07:36:07, user=torsten): Aber sowas von!
      (2023-09-08 07:36:07, user=torsten): Mkay
(2023-10-18 11:00:30, user=torsten): Tach allerseits!
(2023-10-18 14:21:01, user=foobar): <@U0622JWLG49> has joined the channel


Get metadata for your slack channel.

In [6]:
slack.get_conversations_info(slack_channel)

{'id': 'C05RK7JB336',
 'name': 'foo',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'is_mpim': False,
 'is_private': False,
 'created': 1694156357,
 'is_archived': False,
 'is_general': False,
 'unlinked': 0,
 'name_normalized': 'foo',
 'is_shared': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'pending_shared': [],
 'context_team_id': 'T05RA4Y6CT0',
 'updated': 1694156357149,
 'parent_conversation': None,
 'creator': 'U05S5PHRU7J',
 'is_ext_shared': False,
 'shared_team_ids': ['T05RA4Y6CT0'],
 'pending_connected_team_ids': [],
 'is_member': True,
 'last_read': '1694157726.261829',
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This channel is for everything #foo. Hold meetings, share docs, and make decisions together with your team.',
  'creator': 'U05S5PHRU7J',
  'last_set': 1694156357},
 'previous_names': [],
 'num_members': 5}

List all channels.

In [7]:
slack.list_channels()

[{'id': 'C05RGLSNBFV',
  'name': 'random',
  'is_channel': True,
  'is_group': False,
  'is_im': False,
  'is_mpim': False,
  'is_private': False,
  'created': 1694156317,
  'is_archived': False,
  'is_general': False,
  'unlinked': 0,
  'name_normalized': 'random',
  'is_shared': False,
  'is_org_shared': False,
  'is_pending_ext_shared': False,
  'pending_shared': [],
  'context_team_id': 'T05RA4Y6CT0',
  'updated': 1694156317807,
  'parent_conversation': None,
  'creator': 'U05S5PHRU7J',
  'is_ext_shared': False,
  'shared_team_ids': ['T05RA4Y6CT0'],
  'pending_connected_team_ids': [],
  'is_member': False,
  'topic': {'value': '', 'creator': '', 'last_set': 0},
  'purpose': {'value': 'This channel is for... well, everything else. It’s a place for team jokes, spur-of-the-moment ideas, and funny GIFs. Go wild!',
   'creator': 'U05S5PHRU7J',
   'last_set': 1694156317},
  'previous_names': [],
  'num_members': 3},
 {'id': 'C05RK7JB336',
  'name': 'foo',
  'is_channel': True,
  'is_grou